In [43]:

import cv2
import numpy as np

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
out_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

fruit_vegetable_classes = {
    "banana", "apple", "orange", "carrot", "broccoli", 
    "cucumber", "grape", "pineapple", "tomato", "pepper"
}

detected_items = set()
cap = cv2.VideoCapture(0)

def detect_objects(frame):
    h, w = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)

    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            
            if confidence > 0.5 and classes[class_id] in fruit_vegetable_classes:
                
                center_x, center_y, width, height = (detection[0:4] * np.array([w, h, w, h])).astype("int")
                x, y = int(center_x - width / 2), int(center_y - height / 2)
                
                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression to eliminate redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            detected_items.add(classes[class_ids[i]])
            
            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def run_detection():
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break
        
        # Detect objects
        detect_objects(frame)
        
        # Display the resulting frame
        cv2.imshow("Food Detection", frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Clean up
    cap.release()
    cv2.destroyAllWindows()
    return list(detected_items)

# Run detection
print("Starting detection... Press 'q' to quit")
detected_objects = run_detection()
print("Detected Objects:", detected_objects)


Starting detection... Press 'q' to quit
Detected Objects: ['banana']


In [ ]:
import openai
from openai import OpenAI
# Set your API key


client = OpenAI(api_key="API-KEY")


def generate_recipe(ingredients, model="gpt-4o", temperature=0.7):
   """
   Generates a recipe based on the provided ingredients.
   """
   prompt = (
       f"I have the following ingredients: {ingredients}.\n"
       "Please create a recipe that uses these ingredients. "
       "List out the ingredients needed (with quantities if relevant) "
       "and provide detailed steps for cooking."
   )


   response = client.chat.completions.create(
       model=model,
       messages=[
           {"role": "user", "content": prompt}
       ],
       temperature=temperature,
       max_tokens=500
   )


   recipe_text = response.choices[0].message.content
   return recipe_text


if __name__ == "__main__":
#    user_ingredients = input("Enter your ingredients (comma-separated): ")
   recipe = generate_recipe(detected_objects)
   print("\n--- AI-Generated Recipe ---\n")
   print(recipe)



--- AI-Generated Recipe ---

With a single banana, you can create a simple and delicious banana smoothie. Here's a quick recipe:

### Ingredients
- 1 ripe banana
- 1 cup of milk (you can use dairy or any plant-based milk like almond, soy, or oat milk)
- 1 tablespoon of honey or maple syrup (optional, for sweetness)
- 1/2 teaspoon of vanilla extract (optional, for extra flavor)
- A handful of ice cubes

### Instructions
1. **Prepare the Banana**: Peel the ripe banana and break it into smaller chunks. This will make it easier to blend.

2. **Combine Ingredients**: In a blender, combine the banana chunks, 1 cup of milk, 1 tablespoon of honey or maple syrup (if using), 1/2 teaspoon of vanilla extract (if using), and a handful of ice cubes.

3. **Blend**: Blend everything on high speed until smooth and creamy. This should take about 30 seconds to 1 minute, depending on your blender.

4. **Taste and Adjust**: Taste the smoothie to see if it's sweet enough for your liking. If you want it swe